In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from numpy.testing import assert_equal

In [2]:
import os
os.environ['AWS_PROFILE'] = 'default'

In [3]:
from pyspark import SparkConf

In [4]:
conf = (SparkConf()
           .setMaster('local[*]')
           .set('spark.jars.packages',
                'com.amazonaws:aws-java-sdk:1.11.563,org.apache.hadoop:hadoop-aws:3.2.2')
           .set('spark.executor.extraJavaOptions',
                '-Dcom.amazonaws.services.s3.enableV4=true')
           .set('spark.driver.extraJavaOptions',
                '-Dcom.amazonaws.services.s3.enableV4=true')
           .set('fs.s3a.aws.credentials.provider',
                'com.amazonaws.auth.profile.ProfileCredentialsProvider,'
                'org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider'))
sc = SparkContext(conf=conf)

:: loading settings :: url = jar:file:/usr/local/spark-3.1.2-bin-hadoop3.2/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/msds2022/jgacal/.ivy2/cache
The jars for the packages stored in: /home/msds2022/jgacal/.ivy2/jars
com.amazonaws#aws-java-sdk added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-01938296-b81f-44f4-8050-15734dfd4420;1.0
	confs: [default]
	found com.amazonaws#aws-java-sdk;1.11.563 in central
	found com.amazonaws#aws-java-sdk-ioteventsdata;1.11.563 in central
	found com.amazonaws#aws-java-sdk-core;1.11.563 in central
	found commons-logging#commons-logging;1.1.3 in central
	found org.apache.httpcomponents#httpclient;4.5.5 in central
	found org.apache.httpcomponents#httpcore;4.4.9 in central
	found commons-codec#commons-codec;1.10 in central
	found software.amazon.ion#ion-java;1.0.2 in central
	found com.fasterxml.jackson.core#jackson-databind;2.6.7.2 in central
	found com.fasterxml.jackson.core#jackson-annotations;2.6.0 in central
	found com.fasterxml.jackson.core#jackson-c

	found com.amazonaws#aws-java-sdk-elasticache;1.11.563 in central
	found com.amazonaws#aws-java-sdk-elastictranscoder;1.11.563 in central
	found com.amazonaws#aws-java-sdk-ec2;1.11.563 in central
	found com.amazonaws#aws-java-sdk-dynamodb;1.11.563 in central
	found com.amazonaws#aws-java-sdk-budgets;1.11.563 in central
	found com.amazonaws#aws-java-sdk-cloudtrail;1.11.563 in central
	found com.amazonaws#aws-java-sdk-cloudwatch;1.11.563 in central
	found com.amazonaws#aws-java-sdk-logs;1.11.563 in central
	found com.amazonaws#aws-java-sdk-events;1.11.563 in central
	found com.amazonaws#aws-java-sdk-cognitoidentity;1.11.563 in central
	found com.amazonaws#aws-java-sdk-cognitosync;1.11.563 in central
	found com.amazonaws#aws-java-sdk-directconnect;1.11.563 in central
	found com.amazonaws#aws-java-sdk-cloudformation;1.11.563 in central
	found com.amazonaws#aws-java-sdk-cloudfront;1.11.563 in central
	found com.amazonaws#aws-java-sdk-clouddirectory;1.11.563 in central
	found com.amazonaws#a

22/03/07 16:21:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/07 16:21:30 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/07 16:21:30 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
22/03/07 16:21:30 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
22/03/07 16:21:30 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
22/03/07 16:21:30 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port 4045.


In [5]:
spark = SparkSession(sc)

In [6]:
!aws s3 ls s3://helpful-sentences-from-reviews/

2021-05-05 14:26:24          0 README.md
2021-05-05 14:26:23          0 index.html
2021-04-18 13:08:05     525322 test.json
2021-04-18 13:08:23    5258801 train.json


# Problem 2

For this problem, you will be working with the [Helpful Sentences from Reviews](https://registry.opendata.aws/helpful-sentences-from-reviews/) training set.

You may **only** use Apache Spark and the Python Standard Library. You **cannot** use numpy, scipy, pandas or scikit-learn. Do **not** print or display large amount of results. You will get deductions if you make the browser unresponsive.

## Problem 2a [10 pts]

Create a function `count_ratings` that reads `train.json` and returns a Spark DataFrame with columns `rating`, which is the `helpful` value rounded to one decimal place, and `count`, the number of sentences with that rating. Sort by `rating`.

**correction: business rounding dapat**

In [7]:
import pyspark.sql.functions as f

In [8]:
from decimal import Decimal



# df = spark.read.json('s3a://helpful-sentences-from-reviews/train.json')
# (df
#  .withColumn('rating', f.udf(lambda x:float(Decimal(str(x)).quantize(Decimal('1.0'))))(f.col('helpful')))
#  .groupBy('rating')
#  .count()
#  .orderBy('rating')
#  .limit(10)
 
#  .toPandas()
# )

In [9]:
# df = spark.read.json()

In [10]:
from pyspark.sql.functions import udf
@udf
def b_round(x):
    import decimal as d
    return (d.Decimal(str(x)).quantize(d.Decimal('1.0')))

In [11]:
def count_ratings():
    from pyspark.sql.types import FloatType
    return (spark.read.json('s3a://helpful-sentences-from-reviews/train.json')
            
 .withColumn('rating', f.round(b_round(f.col('helpful')), 1))
 .groupBy('rating')
 .count()
#  .withColumn('rating', f.udf(lambda x: x*1.0)(f.col('rating').cast('float')))
 .orderBy('rating')
           )

In [12]:
pdf_ratings_count = count_ratings().toPandas()
assert_equal(len(pdf_ratings_count), 21)
assert_equal(pdf_ratings_count.columns.tolist(), ['rating', 'count'])
assert_equal(
    pdf_ratings_count.iloc[:10].to_numpy().tolist(),
    [[0.0, 37.0],
     [0.1, 91.0],
     [0.2, 188.0],
     [0.3, 272.0],
     [0.4, 452.0],
     [0.5, 615.0],
     [0.6, 797.0],
     [0.7, 970.0],
     [0.8, 1122.0],
     [0.9, 1376.0]]
)

22/03/07 16:21:45 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


## Problem 2b [15 pts]

Cluster the sentences by their `sentence` text using k-means. You may preprocess it however you want. Select the optimal number of clusters using silhouette coefficient. Make sure that the steps that you are doing are sufficiently documented, and the final model and number of clusters are explicit.

In [13]:
df = spark.read.json('s3a://helpful-sentences-from-reviews/train.json')
df.limit(4).toPandas()

,asin,helpful,main_image_url,product_title,sentence
0,B000AO3L84,1.70,http://ecx.images-amazon.com/images/I/41XAEKR9...,Canon 430EX Speedlite Flash for Canon EOS SLR ...,this flash is a superb value.
1,B001SEQPGK,1.30,http://ecx.images-amazon.com/images/I/71KLvmtc...,Sony Cyber-shot DSC-W290 12 MP Digital Camera ...,The pictures were not sharp at all.
2,0553386697,1.90,http://ecx.images-amazon.com/images/I/81HdbmkR...,The Whole-Brain Child: 12 Revolutionary Strate...,A very good resource for parents.
3,B006SUWZH2,0.25,http://ecx.images-amazon.com/images/I/61A2WQOL...,Memorex Portable CD Boombox with AM FM Radio,"We have it in a child's room, and will be swit..."


In [14]:
df_sen = (df
 .select('sentence')
 .dropna()
)

df_sen.limit(4).toPandas()

,sentence
0,this flash is a superb value.
1,The pictures were not sharp at all.
2,A very good resource for parents.
3,"We have it in a child's room, and will be swit..."


In [15]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from pyspark.ml import Pipeline


perform tfidf

In [16]:
tokenizer = Tokenizer(inputCol="sentence", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(),outputCol="rawFeatures", numFeatures=30)
idf = IDF(inputCol=hashingTF.getOutputCol(), outputCol="features",minDocFreq=5)
pipeline = Pipeline(stages=[tokenizer, hashingTF, idf, KMeans(k=8)])

transformer = pipeline.fit(df_sen)
df_transformed = transformer.transform(df_sen)

df_transformed.limit(5).toPandas()

22/03/07 16:22:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
22/03/07 16:22:42 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


,sentence,words,rawFeatures,features,prediction
0,this flash is a superb value.,"[this, flash, is, a, superb, value.]","(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 0.0, 0.7014309842370906, 0.0, 0.0, ...",0
1,The pictures were not sharp at all.,"[the, pictures, were, not, sharp, at, all.]","(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...","(0.0, 0.0, 0.0, 0.0, 0.0, 1.5044524483502613, ...",0
2,A very good resource for parents.,"[a, very, good, resource, for, parents.]","(0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 1.0, ...","(0.0, 0.0, 0.0, 0.0, 1.338888418713637, 1.5044...",0
3,"We have it in a child's room, and will be swit...","[we, have, it, in, a, child's, room,, and, wil...","(1.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 1.0, 0.0, ...","(0.9746321390013013, 0.0, 0.0, 0.0, 1.33888841...",3
4,Again the makers are too lazy to bring in the ...,"[again, the, makers, are, too, lazy, to, bring...","(0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, ...","(0.0, 0.0, 1.6879901393390995, 0.7014309842370...",3


In [17]:
evaluator = ClusteringEvaluator()

perform CV

In [18]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

paramGrid = (ParamGridBuilder()
                .addGrid(pipeline.getStages()[-1].k, [2,3, 4,5,6,7, 8]) #add grid here specificies the variable you want to change
                .build())

cv = CrossValidator(estimator=pipeline,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator)

cv_trained=cv.fit(df_sen)

In [19]:
cv_trained.bestModel.stages[-1]

KMeansModel: uid=KMeans_4a94fc0b4758, k=2, distanceMeasure=euclidean, numFeatures=30

In [20]:
cv_trained.bestModel.stages[-1].explainParam('k')

'k: The number of clusters to create. Must be > 1. (default: 2, current: 2)'

best k is 2. Metric used is silhouette coeff by default.